In [ ]:
import sys
sys.path.append("..")

In [ ]:
import os
import csv
import pickle
import numpy as np
import pandas as pd
import google_reviews
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
path = "../../cleaned_data/tainan/word_segments/"

In [ ]:
# define function to read pickle
def read_pickle(path):
    if path[-4:] != '.pkl':
        raise ValueError
    with open(path, 'rb') as fd:
        f = pickle.load(fd)
    return f

In [ ]:
#define fuction to choose words by pos 
def clear(ckip):
    ckip_ = ckip.copy()
    pos_dic = {"Na", "Nb"}
    wss = []
    poss = []
    for ws, pos in zip(ckip_["ws"], ckip_["pos"]):
        wss_ = []
        poss_ = []
        for i, j in zip(ws, pos):
            if j in pos_dic:
                wss_.append(i)
                poss_.append(j)
        wss.append(wss_)
        poss.append(poss_)
        
    ckip_["ws"] = wss
    ckip_["pos"] = poss
    return ckip_

In [ ]:
# define function to union each word segment result for each shop
def union_ws_result(ckip):
    doc = []
    for ws in ckip["ws"]:
        for i in ws:
            doc.append(i)
    return doc

Get analysis data

In [ ]:
ws_all = {}

# read all pickle files
for filename in os.listdir(path):
    if filename.endswith(".pkl"):
        ckip = read_pickle(os.path.join(path, filename))
        ckip_ = clear(ckip)
        #union result to dictionary
        result = union_ws_result(ckip_)
        ws_all[filename.replace(".pkl", "") ]= result

In [ ]:
ws_all

In [ ]:
#get top N keywords (with term frequency)
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
dic = pd.read_excel("../../dic.xlsx")
df = pd.DataFrame(dic)
four_aspect = df.set_index(['word'])["category"].to_dict()

In [ ]:
category

In [ ]:
#save term frequency to csv file
output_folder = "../../Google_review_code/result/keywords/Tainan/"
for key in ws_all:
    common_words = get_top_n_words(ws_all[key], 20)
    filename = pd.DataFrame(common_words, columns = ['keywords' , 'counts'])
    filename.to_csv(os.path.join(output_folder, key+".csv"), index=False)